# Data Anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage30yr""","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage30yr""","""Unknown""",669,"""Laurianneshire""","""AutoLoans""","""Accepted"""
"""MoneyMarketSavingsAccount""","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmieshire""","""DepositAccounts""","""Accepted"""
"""UPlusFinPersonal""","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccounts""","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
f64,str,str,str,str,str,bool
4.1686e18,"""14637657931422199389""","""../../../../data/SampleHDS.jso…","""3866700847298902154""","""12358087534995492875""","""17564598795835530550""",false
1.3031e19,"""3540750323845704751""","""../../../../data/SampleHDS.jso…","""16490331504929666860""","""12358087534995492875""","""13455572550305862765""",true
5.4573e18,"""12478214287973360110""","""../../../../data/SampleHDS.jso…","""519849813049818649""","""13326932866139446719""","""17564598795835530550""",false
1.5830e19,"""6153035652651966767""","""../../../../data/SampleHDS.jso…","""16490331504929666860""","""7942898883585234477""","""17564598795835530550""",false
2.3474e18,"""14180602842666418032""","""../../../../data/SampleHDS.jso…","""3866700847298902154""","""7942898883585234477""","""254546774000816466""",true
1.7591e19,"""767599322053724639""","""../../../../data/SampleHDS.jso…","""519849813049818649""","""8357854104330504918""",null,false
2.6857e18,"""5139431550260707837""","""../../../../data/SampleHDS.jso…","""519849813049818649""","""7942898883585234477""","""254546774000816466""",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_CLV': 'PREDICTOR_0',
 'Customer_City': 'PREDICTOR_1',
 'filename': 'filename',
 'Customer_MaritalStatus': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
2.8255e18,"""3253716076551053117""","""../../../../data/SampleHDS.jso…","""1607024102700391407""","""FirstMortgage30yr""","""9891089492734519652""","""Rejected"""
5.9692e18,"""1852970730006183898""","""../../../../data/SampleHDS.jso…","""13622254631908955277""","""FirstMortgage30yr""","""3041149141662925473""","""Accepted"""
4.1994e18,"""3817374094926815879""","""../../../../data/SampleHDS.jso…","""13394975159169219223""","""MoneyMarketSavingsAccount""","""9891089492734519652""","""Rejected"""
1.0093e19,"""5551538965335559334""","""../../../../data/SampleHDS.jso…","""13622254631908955277""","""BasicChecking""","""9891089492734519652""","""Rejected"""
1.3826e19,"""14586809871653012017""","""../../../../data/SampleHDS.jso…","""1607024102700391407""","""BasicChecking""","""27620031867254223""","""Accepted"""
6.5678e18,"""18112733665592067882""","""../../../../data/SampleHDS.jso…","""13394975159169219223""","""UPlusFinPersonal""",null,"""Rejected"""
8.4156e18,"""6302897723061124311""","""../../../../data/SampleHDS.jso…","""13394975159169219223""","""BasicChecking""","""27620031867254223""","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
1.3501e19,"""6347825690736596091""","""../../../../data/SampleHDS.jso…","""12102011276940863267""","""FirstMortgage30yr""","""15601228264516668495""","""Rejected"""
4.0944e18,"""9734664799150752577""","""../../../../data/SampleHDS.jso…","""2950847690053844960""","""FirstMortgage30yr""","""7642952337112879926""","""Accepted"""
9.5842e17,"""7084410200669564064""","""../../../../data/SampleHDS.jso…","""5240500049675399139""","""MoneyMarketSavingsAccount""","""15601228264516668495""","""Rejected"""
1.2500e19,"""11217605736643152697""","""../../../../data/SampleHDS.jso…","""2950847690053844960""","""BasicChecking""","""15601228264516668495""","""Rejected"""
6.1603e18,"""13670618154428111670""","""../../../../data/SampleHDS.jso…","""12102011276940863267""","""BasicChecking""","""11131091301139825299""","""Accepted"""
5.7730e18,"""5629208720009966647""","""../../../../data/SampleHDS.jso…","""5240500049675399139""","""UPlusFinPersonal""",null,"""Rejected"""
2.3751e18,"""17459191497894494987""","""../../../../data/SampleHDS.jso…","""5240500049675399139""","""BasicChecking""","""11131091301139825299""","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_CLV=PREDICTOR_0
Customer_City=PREDICTOR_1
filename=filename
Customer_MaritalStatus=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_1,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str
1.7992e18,"""14138276159533664991""","""6765603423719828582""","""FirstMortgage30yr""","""15951300822218250805""","""Rejected"""
1.3901e19,"""11332387639068343098""","""1928326046389916721""","""FirstMortgage30yr""","""4464183993344040734""","""Accepted"""
1.5921e19,"""15037561614734899504""","""4252301341901276901""","""MoneyMarketSavingsAccount""","""15951300822218250805""","""Rejected"""
1.1602e19,"""1303572114386445365""","""1928326046389916721""","""BasicChecking""","""15951300822218250805""","""Rejected"""
7.4361e18,"""14217832479365246719""","""6765603423719828582""","""BasicChecking""","""13198385440936288050""","""Accepted"""
1.1067e19,"""11748335234070004762""","""4252301341901276901""","""UPlusFinPersonal""",null,"""Rejected"""
2.4172e18,"""419572297372726588""","""4252301341901276901""","""BasicChecking""","""13198385440936288050""","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.



ComputeError: AttributeError: 'int' object has no attribute 'encode'